# FEATURE EXTRACTION

In [27]:
# you start empty df
# loop over mt files
# create subdf that has only movement for the particular tier
# and then you calculate for this file whatever you want
# and append it to the empty df with the trialID 
import os
import glob
import pandas as pd
import numpy as np
import math

import scipy
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

import math

import xml.etree.ElementTree as ET


curfolder = os.getcwd()
print(curfolder)

mtfolder = curfolder + '/TS_movement/'
print(mtfolder)

mtfiles = glob.glob(mtfolder + '*.csv')

demographics = curfolder + "/LNDW.csv"



# create an empty dataframe where I will store all of the features.



/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_movement/


In [28]:
# gesture space calculator: 

def gesturespace(sample, joint, side, movement_type): 
    fileID = sample["fileID"]
    fileID = fileID.replace('_comr.mp4', '')
    #get the subdf for where there is only arm movement. 
    total_movement = int(0)
    movement_sample = sample[sample[movement_type] == 'movement']
    X_std = np.std(movement_sample["X_" + side + "_"+ joint])
    Y_std = np.std(movement_sample["Y_" + side + "_"+ joint])
    Z_std = np.std(movement_sample["Z_" + side + "_"+ joint])
    total_movement = X_std + Y_std + Z_std
    return total_movement



In [29]:
# creates the dataframe of features
featuredf = pd.DataFrame()

# gets the survey information
demographics_read = pd.read_csv(demographics)

# adds survey columns to the features csv file
columns_to_add = demographics_read.columns
for col in columns_to_add:
    featuredf[col] = pd.NA


# Big For Loop For Feature Extraction

In [ ]:
# big loop to loop through all of the files and extract features


count = 0
for file in mtfiles:
    print(file)
    sample = pd.read_csv(file)

    # get the fileID 
    featuredf.loc[count, "fileID"] = (sample["fileID"])[0]

    # get the chainID and the participantID
    featuredf.loc[count, "chainID"] = sample["participantID"][0]
    chainID = featuredf["chainID"][count]
    split_string = chainID.split('ch')[1]
    split_string = split_string.split('g')
    chain_num = split_string[0]
    gen_num = split_string[1]
    participantID = int(chain_num) * int(gen_num)
    featuredf.loc[count, "participantID"] = participantID

    # get the demographic information from the survey and puts it in correct row in featuredf 
    matching_rows = demographics_read[demographics_read["Q1"] == str(int(participantID))]
    if len(matching_rows) >0:
        demographic_row = matching_rows.iloc[0]
        matching_row = demographics_read[demographics_read["Q1"] == str(int(participantID))]
        featuredf.loc[count, columns_to_add] = matching_row.iloc[0].values
    else:
        print("no matching demographics row")

    #left arm
    featuredf["left_index_movement"] = gesturespace(sample, "INDEX", "LEFT", "arms")
    featuredf["left_wrist_movement"] = gesturespace(sample, "WRIST", "LEFT", "arms")
    featuredf["left_elbow_movement"] = gesturespace(sample, "ELBOW", "LEFT", "arms")
    featuredf["total_left_arm_movement"] = featuredf["left_index_movement"] + featuredf["left_wrist_movement"] + featuredf["left_elbow_movement"]

    # right arm
    featuredf["right_index_movement"] = gesturespace(sample, "INDEX", "RIGHT", "arms")
    featuredf["right_wrist_movement"] = gesturespace(sample, "WRIST", "RIGHT", "arms")
    featuredf["right_elbow_movement"] = gesturespace(sample, "ELBOW", "RIGHT", "arms")
    featuredf["total_right_arm_movement"] = featuredf["right_index_movement"] + featuredf["right_wrist_movement"] + featuredf["right_elbow_movement"]

    # left leg
    featuredf["left_ankle_movement"] = gesturespace(sample, "ANKLE", "LEFT", "arms")
    featuredf["left_wrist_movement"] = gesturespace(sample, "KNEE", "LEFT", "arms")
    featuredf["left_elbow_movement"] = gesturespace(sample, "HIP", "LEFT", "arms")
    featuredf["total_left_arm_movement"] = featuredf["left_index_movement"] + featuredf["left_wrist_movement"] + featuredf["left_elbow_movement"]

    count = count +1
    break

In [ ]:
featuredf

# feature extraction for a single file

In [30]:
file = "/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_movement/Donner_g_ch2_g1_compr.eaf.csv"

count = 1
# get the sample
print(file)
sample = pd.read_csv(file)

# get the fileID 
featuredf.loc[count, "fileID"] = (sample["fileID"])[0]

# get the chainID and the participantID
featuredf.loc[count, "chainID"] = sample["participantID"][0]
chainID = featuredf["chainID"][count]
split_string = chainID.split('ch')[1]
split_string = split_string.split('g')
chain_num = split_string[0]
gen_num = split_string[1]
participantID = int(chain_num) * int(gen_num)
featuredf.loc[count, "participantID"] = participantID

# get the demographic information from the survey and puts it in correct row in featuredf 
matching_rows = demographics_read[demographics_read["Q1"] == str(int(participantID))]
if len(matching_rows) >0:
    demographic_row = matching_rows.iloc[0]
    matching_row = demographics_read[demographics_read["Q1"] == str(int(participantID))]
    featuredf.loc[count, columns_to_add] = matching_row.iloc[0].values
else:
    print("no matching demographics row")


# gesture space processing

#left arm
featuredf["left_index_movement"] = gesturespace(sample, "INDEX", "LEFT", "arms")
featuredf["left_wrist_movement"] = gesturespace(sample, "WRIST", "LEFT", "arms")
featuredf["left_elbow_movement"] = gesturespace(sample, "ELBOW", "LEFT", "arms")
featuredf["total_left_arm_movement"] = featuredf["left_index_movement"] + featuredf["left_wrist_movement"] + featuredf["left_elbow_movement"]

# right arm
featuredf["right_index_movement"] = gesturespace(sample, "INDEX", "RIGHT", "arms")
featuredf["right_wrist_movement"] = gesturespace(sample, "WRIST", "RIGHT", "arms")
featuredf["right_elbow_movement"] = gesturespace(sample, "ELBOW", "RIGHT", "arms")
featuredf["total_right_arm_movement"] = featuredf["right_index_movement"] + featuredf["right_wrist_movement"] + featuredf["right_elbow_movement"]

# left leg
featuredf["left_ankle_movement"] = gesturespace(sample, "ANKLE", "LEFT", "lower_body")
featuredf["left_knee_movement"] = gesturespace(sample, "KNEE", "LEFT", "lower_body")
featuredf["left_hip_movement"] = gesturespace(sample, "HIP", "LEFT", "lower_body")
featuredf["total_left_leg_movement"] = featuredf["left_ankle_movement"] + featuredf["left_knee_movement"] + featuredf["left_hip_movement"]





/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_movement/Donner_g_ch2_g1_compr.eaf.csv


In [31]:
featuredf

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,left_elow_movement,total_left_arm_movement,right_index_movement,right_wrist_movement,right_elow_movement,total_right_arm_movement,left_ankle_movement,left_knee_movement,left_hip_movement,total_left_leg_movement
1,2024-06-22 17:14:18,2024-06-22 17:16:26,IP Address,109.40.242.194,100,127,True,2024-06-22 17:16:26,R_2KIpvLjRioa2KCq,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [ ]:
total_movement_index = gesturespace(file, "INDEX", "LEFT", "arms")
total_movement_wrist = gesturespace(file, "WRIST", "LEFT", "arms")
total_movement_elbow = gesturespace(file, "ELBOW", "LEFT", "arms")
print(total_movement_index)
print(total_movement_wrist)
print(total_movement_elbow)
total_arm_movement = total_movement_index + total_movement_wrist + total_movement_elbow
print(total_arm_movement)


In [35]:
featuredf

,fileID,chainID,participantID
0,Donner_g_ch1_g10_compr.mp4,ch1g10,10.0
1,Donner_g_ch2_g10_compr.mp4,ch2g10,20.0
2,Donner_g_ch3_g10_compr.mp4,ch3g10,30.0
